In [2]:
# Importing all the libraries necessary for the project
import jax
import flax
import numpy as np
import jax.numpy as jnp
import tensorflow as tf
from jax import random
import pandas as pd
import os
from flax import linen as nn # the Linen API
from flax.training import train_state 
import optax
import matplotlib.pyplot as plt
%matplotlib inline

2025-06-09 09:00:10.687374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749459610.977280      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749459611.057126      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# List all the available devices
jax.local_devices()


INFO:2025-06-09 09:00:26,817:jax._src.xla_bridge:924: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:2025-06-09 09:00:26,836:jax._src.xla_bridge:924: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


[CpuDevice(id=0)]

In [4]:
mnist_train = pd.read_csv('../input/digit-recognizer/train.csv')

In [5]:
mnist_train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
class FfwModel():
    def __init__(self, n_x, n_y, key):
        self.n_x = n_x
        self.n_y = n_y
        k1, k2 = random.split(key)
        self.W1 = random.uniform(k1, (n_x, 100)) * 0.01
        self.W2 = random.uniform(k2, (100, n_y)) * 0.01

    def forward(self, X_batch, y_batch=None):
        Z1 = jnp.dot(self.W1.T, X_batch)
        A1 = jax.nn.relu(Z1)
        Z2 = jnp.dot(self.W2.T, A1)
        A2 = 1 / (1 + jnp.exp(-Z2))

        if(y_batch is not None):
            # cross entropy
            one_hot_labels = jax.nn.one_hot(y_batch, num_classes=10).T.reshape(10,y_batch.shape[1])
            nll = -(one_hot_labels * jnp.log(A2) + (1-one_hot_labels) * jnp.log(1-A2))
            cost = jnp.mean(jnp.sum(nll, axis=-1))
            # print(nll[:, :3])
            # print(A2[:, :3])
            # print(y_batch[:, :3])
            # print(jnp.argmax(A2, axis=0) == y_batch)
            accuracy = jnp.mean(jnp.argmax(A2, axis=0) == y_batch)
            self.Z1 = Z1
            return A2, A1, cost, accuracy

        return A2, A1, None, None


    def backward(self, A2, A1, A0, y_batch, learning_rate=0.003):
        dL2 = (1/y_batch.shape[1])
        one_hot_labels = jax.nn.one_hot(y_batch, num_classes=10).T.reshape(10,y_batch.shape[1])
        # dA2 = ((A2-one_hot_labels)/(A2*(1-A2))) * dL2
        # dZ2 = dA2 * (A2*(1-A2)) 
        dZ2 = dL2 * (A2-one_hot_labels)
        dW2 = jnp.dot(A1, dZ2.T) 
        dA1 = jnp.dot(self.W2, dZ2)
        dZ1 = self.relu_derivative(self.Z1) * dA1
        dW1 = jnp.dot(A0, dZ1.T) 
        self.W1 -= (learning_rate*dW1)
        self.W2 -= (learning_rate*dW2)
        

    def relu_derivative(self, x):
        return np.where(x >= 0, 1, 0)
        

In [7]:
labels = mnist_train["label"]
features = mnist_train.drop(labels = ["label"],axis = 1)
features = features/255.0
labels = labels.to_numpy().reshape(1, -1)
features = features.values.T

In [8]:
print(features.shape)
print(labels.shape)


(784, 42000)
(1, 42000)


In [ ]:
epochs = 200
batch_size = 100
rng = jax.random.PRNGKey(7)
rng, init_rng = jax.random.split(rng)
model = FfwModel(features.shape[0], 10, rng)

for epoch in range(epochs):
    cost = None
    accuracy = None
    for step in range(features.shape[1]//batch_size):
        start = step*batch_size
        end = ((step+1)*batch_size)
        X_batch = features[:, start:end]
        y_batch = labels[:, start:end]
        A2, A1, cost, accuracy = model.forward(X_batch, y_batch)
        model.backward(A2, A1, X_batch, y_batch, 0.003)
    print(f'epoch: {epoch} cost: {cost} train_acc:{accuracy*100}')

epoch: 0 cost: 34.44282913208008 train_acc:4.999999523162842
epoch: 1 cost: 34.15329360961914 train_acc:4.999999523162842
epoch: 2 cost: 33.80289840698242 train_acc:14.999999046325684
epoch: 3 cost: 33.03246307373047 train_acc:16.0
epoch: 4 cost: 31.11858558654785 train_acc:55.0
epoch: 5 cost: 27.856369018554688 train_acc:63.0
epoch: 6 cost: 24.50994300842285 train_acc:66.99999237060547
epoch: 7 cost: 21.900693893432617 train_acc:69.0
epoch: 8 cost: 19.928089141845703 train_acc:74.0
epoch: 9 cost: 18.34296989440918 train_acc:77.0
epoch: 10 cost: 16.97061538696289 train_acc:78.0
epoch: 11 cost: 15.753348350524902 train_acc:78.0
epoch: 12 cost: 14.67704963684082 train_acc:79.99999237060547
epoch: 13 cost: 13.741900444030762 train_acc:82.0
epoch: 14 cost: 12.940051078796387 train_acc:83.0
epoch: 15 cost: 12.26222038269043 train_acc:85.99999237060547
epoch: 16 cost: 11.685172080993652 train_acc:85.0
epoch: 17 cost: 11.18864917755127 train_acc:85.0
epoch: 18 cost: 10.751402854919434 train_a

In [10]:
A2, _, _, _ = model.forward(features[:, 700])
print(A2)
print(jnp.argmax(A2, axis=0))
print(labels[0,700])

[1.8518037e-04 5.8256609e-07 8.2342438e-03 3.9470393e-05 9.9384272e-01
 1.8130058e-02 7.1979505e-03 3.2741819e-03 1.7154013e-04 4.0781719e-04]
4
4


In [11]:
test = pd.read_csv('../input/digit-recognizer/test.csv')


In [19]:
test = test
features = test.to_numpy().T
predictions, _, _, _ = model.forward(features)
print(predictions.shape)
labels = jnp.argmax(predictions, axis=0)
print(labels)

(10, 28000)
[2 0 9 ... 3 9 2]


In [25]:
submission = pd.DataFrame( {"ImageId":test.index+1 , "Label": labels} ) #! creat a Taple for Ruslt 
submission.to_csv('/kaggle/working/submission.csv' , index= False)
submission.head(10)

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3
